# 元类

## 类也是对象

- **在 Python 中一切皆对象，类也属于对象的一种。**
- 通常情况下，"普通类(class)"可以用来生成实例(instance)，同样的，"元类 (meta-class)"也可以生成实例，生成的实例就是"普通类"了
- 对象的类型叫作类(class)，类的类型就称作元类 meta-class

- ![形象的说明了，对象，类，元类的关系](https://pic4.zhimg.com/v2-65d2a7c41f6962eed773c6ca9c45f2d5_1200x500.jpg)

## 动态的创建类

- 因为类也是对象，你可以在运⾏时动态的创建它们，就像其他任何对象⼀样。

In [ ]:
def choose_class(name):
    
    if name == "foo":
        class Foo:
            pass
        return Foo
    elif name == "bar":
        class Bar:
            pass
        return Bar

# 执行函数，当传入参数不同时，创建的类也不同
myclass = choose_class("foo")
obj = myclass()

print(myclass)
print(obj)



## 使用type创建类

- 上面的动态创建类不够灵活
- type还有⼀种完全不同的功能，动态的创建类。type可以接受⼀个类的描述作为参数，然后返回⼀个类。
- type(类名, 由⽗类名称组成的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)
- 注意：
    - type的第2个参数，元组中是⽗类的名字，⽽不是字符串

In [ ]:
# 使用type() 创建类
test1 = type('Test1', (), {'num': 100})

help(test1)

t = test1()
print(t.num)

In [ ]:
# 使用class关键字创建类
class Test2(object):
    num = 200

help(Test2)

t = Test2()

print(t.num)

- 通过上面两个例子可以发现，使用 type() 创建类 和使用 class 关键字创建类的效果是一样的。

## 使用 type 创建带有属性和方法的类

- type 接受⼀个字典来为类定义属性，添加的属性是类属性，并不是实例属性。
- 使用 type 为类增加⽅法。只需要定义⼀个有着恰当签名的函数并将其作为属性赋值就可以了。
- 添加属性和方法的方式类似：
    - `{'attr_name': attr_value, 'func_name': func_name}`


In [ ]:
def __init__(self, name):
    self.name = name
    self.__class__.count += 1

def get_name(self):
    return self.name

@classmethod
def get_count(cls):
    """获取该类实例化对象的个数"""
    return cls.count

@staticmethod
def func():
    print('---这是一个静态方法---')
    
    
Test = type('Test', (), {'count': 0, "__init__": __init__, 'get_name': get_name, "get_count": get_count, 'func': func})

t1 = Test('zyp')
t2 = Test('pyz')
t3 = Test('z')

help(Test)
print("Test 实例化对象个数为：", Test.get_count())
print(t1.name)
print(t1.get_name())
t1.func()

print(hasattr(Test, 'get_name'))


## 到底什么是元类

- 元类就是⽤来创建类的"东⻄", 元类就是类的类
- Python中所有的东⻄，注意，我是指所有的东⻄——都是对象。这包括整数、字符串、函数以及类
- 函数type实际上是⼀个元类，type就是Python在背后⽤来创建所有类的元类
- `__class__`是类的一个内置属性，也是每个类实例的，它是一个类的引用, 可以使用 `object.__class__`来查看对象是哪个类的实例

In [ ]:
age = 20

class Test(object):
    pass

t = Test()

print(age.__class__)
print(t.__class__)
print(Test.__class__)

- 可以看到，自定义的类是 type 类的实例

## `__metaclass__`属性

- type就是Python的内建元类，如果需要，我们也可以创建⾃⼰的元类。
- 创建自己的元类需要借用 `__metaclass__`属性

In [ ]:
class Foo(object):
    __metaclass__ = xxxxx
    ........

- 如果你这么做了，Python就会⽤元类来创建类Foo。⼩⼼点，这⾥⾯有些技巧。你⾸先写下class Foo(object)，但是类Foo还没有在内存中创建。Python会在类的定义中寻找__metaclass__属性，如果找到了，Python就会⽤它来创建类Foo，如果没有找到，就会⽤内建的type来创建这个类。

- Python做了如下的操作：
    1. Foo中有__metaclass__这个属性吗？如果是，Python会通过`__metaclass__`创建⼀个名字为Foo的类(对象)
    2. 如果Python没有找到`__metaclass__`，它会继续在 ⽗类 中寻找`__metaclass__`属性，并尝试做和前⾯同样的操作。
    3. 如果Python在任何⽗类中都找不到`__metaclass__`，它就会在模块层次中去寻找`__metaclass__`，并尝试做同样的操作。
    4. 如果还是找不到`__metaclass__`,Python就会⽤内置的type来创建这个类对象。

## 自定义元类

- 元类的主要⽬的就是为了当创建类时能够⾃动地改变类
- 下面是一个例子：通过自定义元类，让模块⾥所有的类的属性都应该是⼤写形式

In [58]:
def upper_attr(class_name, class_parents, class_attr):
    new_attr = {}
    
    for k, v in class_attr.items():
        if not k.startswith('__'):
            new_attr[k.upper()] = v
    
    return type(class_name, class_parents, new_attr)


class Foo(object, metaclass=upper_attr):
    age = 20
    name = 'asd'

f = Foo()

print(hasattr(f, "age"))
print(hasattr(f, "name"))

print(hasattr(f, "AGE"))
print(f.AGE, f.NAME)


False
False
True
20 asd


- 上面是一个使用函数来当元类的例子，下面是使用class来当元类

In [63]:
class UpperAttr(type):
    # __new__ 是在__init__之前被调⽤的特殊⽅法
    # __new__是⽤来创建对象并返回之的⽅法
    def __new__(cls, class_name, class_parents, class_attr):
        
        new_attr = {}
        
        for k, v in class_attr.items():
            if not k.startswith('__'):
                new_attr[k.upper()] = v
        
        return super(UpperAttr, cls).__new__(cls, class_name, class_parents, new_attr)

    
class Foo(object, metaclass=UpperAttr):
    age = 20
    name = 'xxx'
    

f = Foo()

print(hasattr(f, 'AGE'))
print(hasattr(f, 'NAME'))

dir(f)

True
True


['AGE',
 'NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

## 元类的作用

1. 拦截类的创建
2. 修改类
3. 返回修改之后的类

## 为什么要使用元类

- "元类就是深度的魔法，99%的⽤户应该根本不必为此操⼼。如果你想搞清楚究竟是否需要⽤到元类，那么你就不需要它。那些实际⽤到元类的⼈都⾮常清楚地知道他们需要做什么，⽽且根本不需要解释为什么要⽤元类。" ——Python界的领袖 Tim Peters